In [1]:
# Data
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import impl.layer as l

# Dataset preparation and pre-processing
mnist = input_data.read_data_sets('data/MNIST_data/', one_hot=False)

X_train, y_train = mnist.train.images, mnist.train.labels
X_val, y_val = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels
# y_test.shape, y_val.shape, y_train.shape
# X_train.shape, X_train.dtype, X_val.shape, X_val.dtype, X_test.shape, X_test.dtype

Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# Pre-processing: normalizing
def normalize(X):
    # max scale for images 255= 2**8= 8 bit grayscale for each channel
    return (X - X.mean(axis=0)) #/ X.std(axis=0)

X_train, X_val, X_test = normalize(X=X_train), normalize(X=X_val), normalize(X=X_test)

In [3]:
# Model
import impl.layer as l # or from impl.layer import *
from impl.loss import * # import all functions from impl.loss file # import impl.loss as loss_func
from sklearn.utils import shuffle as skshuffle

class FFNN:

    def __init__(self, D, C, H, L):
        self.L = L # number of layers or depth
        self.losses = {'train':[], 'valid':[], 'valid_acc':[]}
        
        # The adaptive/learnable/updatable random feedforward
        self.model = []
        self.W_fixed = []
        self.grads = []
        self.ys_prev = []
        low, high = -1, 1
        
        # Input layer: weights/ biases
        m = dict(W=np.random.uniform(size=(D, H), low=low, high=high) / np.sqrt(D / 2.), b=np.zeros((1, H)))
        self.model.append(m)
        # Fixed feedback weight
        m = np.random.uniform(size=(D, H), low=low, high=high) / np.sqrt(D / 2.)
        self.W_fixed.append(m)
        # Input layer: gradients
        self.grads.append({key: np.zeros_like(val) for key, val in self.model[0].items()})
        # Previous output layer
        self.ys_prev.append(0.0)

        # Hidden layers: weights/ biases
        m_L = []
        for _ in range(L):
            m = dict(W=np.random.uniform(size=(H, H), low=low, high=high) / np.sqrt(H / 2.), b=np.zeros((1, H)))
            m_L.append(m)
        self.model.append(m_L)
        # Fixed feedback weight
        m_L = []
        for _ in range(L):
            m = np.random.uniform(size=(H, H), low=low, high=high) / np.sqrt(H / 2.)
            m_L.append(m)
        self.W_fixed.append(m_L)
        # Hidden layer: gradients
        grad_L = []
        for _ in range(L):
            grad_L.append({key: np.zeros_like(val) for key, val in self.model[1][0].items()})
        self.grads.append(grad_L)
        # Previous output layer
        ys_prev_L = []
        for _ in range(L):
            ys_prev_L.append(0.0)
        self.ys_prev.append(ys_prev_L)
        
        # Output layer: weights/ biases
        m = dict(W=np.random.uniform(size=(H, C), low=low, high=high) / np.sqrt(H / 2.), b=np.zeros((1, C)))
        self.model.append(m)
        # Fixed feedback weight
        m = np.random.uniform(size=(H, C), low=low, high=high) / np.sqrt(H / 2.)
        self.W_fixed.append(m)
        # Outout layer: gradients
        self.grads.append({key: np.zeros_like(val) for key, val in self.model[2].items()})
        # Previous output layer
        self.ys_prev.append(0.0)
        
    def fc_forward(self, X, W, b):
        out = (X @ W) + b
        cache = (W, X)
        return out, cache

    def fc_backward(self, dout, cache, W_fixed):
        W, X = cache

        dW = X.T @ dout
        db = np.sum(dout, axis=0).reshape(1, -1) # db_1xn
        
#         dX = dout @ W.T # Backprop
        dX = dout @ W_fixed.T # fb alignment

        return dX, dW, db

    def train_forward(self, X, train):
        caches, ys = [], []
        
        # Input layer
        y, fc_cache = self.fc_forward(X=X, W=self.model[0]['W'], b=self.model[0]['b']) # X_1xD, y_1xc
        y, nl_cache = l.tanh_forward(X=y)
        if train:
            caches.append((fc_cache, nl_cache))
        ys.append(y) # ys[0]
        X = y.copy() # pass to the next layer
        
        # Hidden layers
        fc_caches, nl_caches, ys_L = [], [], []
        for layer in range(self.L):
            y, fc_cache = self.fc_forward(X=X, W=self.model[1][layer]['W'], b=self.model[1][layer]['b'])
            y, nl_cache = l.tanh_forward(X=y)
            ys_L.append(y) # ys[1][layer]
            X = y.copy() # pass to next layer
            if train:
                fc_caches.append(fc_cache)
                nl_caches.append(nl_cache)
        if train:
            caches.append((fc_caches, nl_caches)) # caches[1]            
        ys.append(ys_L) # ys[1]            
        
        # Output layer
        y, fc_cache = self.fc_forward(X=X, W=self.model[2]['W'], b=self.model[2]['b'])
        if train:
            caches.append(fc_cache)
        ys.append(y) # ys[2]

        return ys, caches # for backpropating the error

    def loss_function(self, y, y_train):
        
        loss = cross_entropy(y, y_train) # softmax is included
        dy = dcross_entropy(y, y_train) # dsoftmax is included
        
        return loss, dy
        
    def train_backward(self, dy, caches, ys):
        grads, ys_prev = self.grads, self.ys_prev # initialized by Zero in every iteration/epoch
        
        # Output layer
        fc_cache = caches[2]
        dX, dW, db = self.fc_backward(dout=dy, cache=fc_cache, W_fixed=self.W_fixed[2])
        dy = dX.copy()
        grads[2]['W'] = dW
        grads[2]['b'] = db

        # Hidden layer
        fc_caches, nl_caches = caches[1]
        for layer in reversed(range(self.L)):
            dy *= ys[1][layer] - ys_prev[1][layer] # temporal diff instead of differentiable function
#             dy = l.tanh_backward(cache=nl_caches[layer], dout=dy) # diffable function
            dX, dW, db = self.fc_backward(dout=dy, cache=fc_caches[layer], W_fixed=self.W_fixed[1][layer])
            dy = dX.copy()
            grads[1][layer]['W'] = dW
            grads[1][layer]['b'] = db
        
        # Input layer
        fc_cache, nl_cache = caches[0]
        dy *= ys[0] - ys_prev[0] # temporal diff instead of differentiable function
#         dy = l.tanh_backward(cache=nl_cache, dout=dy) # diffable function
        dX, dW, db = self.fc_backward(dout=dy, cache=fc_cache, W_fixed=self.W_fixed[0])
        grads[0]['W'] = dW
        grads[0]['b'] = db

        return dX, grads
    
    def test(self, X):
        ys_logit, _ = self.train_forward(X, train=False)
        y_logit = ys_logit[2] # last layer
        
        # if self.mode == 'classification':
        y_prob = l.softmax(y_logit) # for accuracy == acc
        y_pred = np.argmax(y_prob, axis=1) # for loss ==err
        
        return y_pred, y_logit
        
    def get_minibatch(self, X, y, minibatch_size, shuffle):
        minibatches = []

        if shuffle:
            X, y = skshuffle(X, y)

        for i in range(0, X.shape[0], minibatch_size):
            X_mini = X[i:i + minibatch_size]
            y_mini = y[i:i + minibatch_size]
            minibatches.append((X_mini, y_mini))

        return minibatches

    def sgd(self, train_set, val_set, alpha, mb_size, n_iter, print_after):
        X_train, y_train = train_set
        X_val, y_val = val_set

        # Epochs
        for iter in range(1, n_iter + 1):

            # Minibatches
            minibatches = self.get_minibatch(X_train, y_train, mb_size, shuffle=True)
            idx = np.random.randint(0, len(minibatches))
            X_mini, y_mini = minibatches[idx]
            
            # Train the model
            ys, caches = self.train_forward(X_mini, train=True)
            loss, dy = self.loss_function(ys[2], y_mini)
            _, grads = self.train_backward(dy, caches, ys) # ys[0], ys[1] and ys_prev are used for backprop
            self.ys_prev = ys # for next iteration or epoch learning dW and db
            self.losses['train'].append(loss)
            
            # Update the model for input layer
            for key in grads[0].keys():
                self.model[0][key] -= alpha * grads[0][key]

            # Update the model for the hidden layers
            for layer in range(self.L):
                for key in grads[1][layer].keys():
                    self.model[1][layer][key] -= alpha * grads[1][layer][key]

            # Update the model for output layer
            for key in grads[2].keys():
                self.model[2][key] -= alpha * grads[2][key]
                
            # Validate the updated model
            y_pred, y_logit = self.test(X_val)
            valid_loss, _ = self.loss_function(y_logit, y_val) # softmax is included in entropy loss function
            self.losses['valid'].append(valid_loss)
            valid_acc = np.mean(y_pred == y_val) # confusion matrix
            self.losses['valid_acc'].append(valid_acc)
            
            # Print the model info: loss & accuracy or err & acc
            if iter % print_after == 0:
                print('Iter-{} train loss: {:.4f} valid loss: {:.4f}, valid accuracy: {:.4f}'.format(
                    iter, loss, valid_loss, valid_acc))

        # Test the final model
        y_pred, y_logit = nn.test(X_test)
        loss, _ = self.loss_function(y_logit, y_test) # softmax is included in entropy loss function
        acc = np.mean(y_pred == y_test)
        print('Last iteration - Test accuracy mean: {:.4f}, std: {:.4f}, loss: {:.4f}'.format(
            acc.mean(), acc.std(), loss))

In [ ]:
# Hyper-parameters
n_iter = 1000000 # number of epochs
alpha = 1e-3 # learning_rate
mb_size = 50 # 2**10==1024 # width, timestep for sequential data or minibatch size
print_after = 100 # n_iter//10 # print loss for train, valid, and test
num_hidden_units = 32 # number of kernels/ filters in each layer
num_input_units = X_train.shape[1] # noise added at the input lavel as input noise we can use dX or for more improvement
num_output_units = y_train.max() + 1 # number of classes in this classification problem
num_layers = 2 # depth 

# Build the model/NN and learn it: running session.
nn = FFNN(C=num_output_units, D=num_input_units, H=num_hidden_units, L=num_layers)

nn.sgd(train_set=(X_train, y_train), val_set=(X_val, y_val), mb_size=mb_size, alpha=alpha, 
           n_iter=n_iter, print_after=print_after)

Iter-100 train loss: 2.2902 valid loss: 2.2901, valid accuracy: 0.1360
Iter-200 train loss: 2.2858 valid loss: 2.2894, valid accuracy: 0.1370
Iter-300 train loss: 2.2759 valid loss: 2.2886, valid accuracy: 0.1390
Iter-400 train loss: 2.3064 valid loss: 2.2879, valid accuracy: 0.1402
Iter-500 train loss: 2.2864 valid loss: 2.2871, valid accuracy: 0.1422
Iter-600 train loss: 2.2663 valid loss: 2.2864, valid accuracy: 0.1430
Iter-700 train loss: 2.2861 valid loss: 2.2856, valid accuracy: 0.1452
Iter-800 train loss: 2.2894 valid loss: 2.2849, valid accuracy: 0.1464
Iter-900 train loss: 2.2895 valid loss: 2.2842, valid accuracy: 0.1468
Iter-1000 train loss: 2.2675 valid loss: 2.2834, valid accuracy: 0.1488
Iter-1100 train loss: 2.3248 valid loss: 2.2826, valid accuracy: 0.1506
Iter-1200 train loss: 2.2905 valid loss: 2.2819, valid accuracy: 0.1512
Iter-1300 train loss: 2.2785 valid loss: 2.2811, valid accuracy: 0.1526
Iter-1400 train loss: 2.2842 valid loss: 2.2804, valid accuracy: 0.1534
I

Iter-11500 train loss: 2.2216 valid loss: 2.2096, valid accuracy: 0.3096
Iter-11600 train loss: 2.2263 valid loss: 2.2089, valid accuracy: 0.3108
Iter-11700 train loss: 2.2379 valid loss: 2.2083, valid accuracy: 0.3116
Iter-11800 train loss: 2.1800 valid loss: 2.2076, valid accuracy: 0.3136
Iter-11900 train loss: 2.2168 valid loss: 2.2070, valid accuracy: 0.3150
Iter-12000 train loss: 2.2151 valid loss: 2.2063, valid accuracy: 0.3164
Iter-12100 train loss: 2.1815 valid loss: 2.2056, valid accuracy: 0.3178
Iter-12200 train loss: 2.2177 valid loss: 2.2050, valid accuracy: 0.3208
Iter-12300 train loss: 2.1925 valid loss: 2.2043, valid accuracy: 0.3216
Iter-12400 train loss: 2.2278 valid loss: 2.2036, valid accuracy: 0.3226
Iter-12500 train loss: 2.2039 valid loss: 2.2030, valid accuracy: 0.3242
Iter-12600 train loss: 2.2221 valid loss: 2.2023, valid accuracy: 0.3250
Iter-12700 train loss: 2.2446 valid loss: 2.2017, valid accuracy: 0.3250
Iter-12800 train loss: 2.1821 valid loss: 2.2010, v

Iter-22800 train loss: 2.0936 valid loss: 2.1382, valid accuracy: 0.3970
Iter-22900 train loss: 2.1216 valid loss: 2.1376, valid accuracy: 0.3968
Iter-23000 train loss: 2.0918 valid loss: 2.1371, valid accuracy: 0.3972
Iter-23100 train loss: 2.1565 valid loss: 2.1365, valid accuracy: 0.3976
Iter-23200 train loss: 2.1508 valid loss: 2.1358, valid accuracy: 0.3974
Iter-23300 train loss: 2.1325 valid loss: 2.1352, valid accuracy: 0.3978
Iter-23400 train loss: 2.1525 valid loss: 2.1346, valid accuracy: 0.3974
Iter-23500 train loss: 2.1428 valid loss: 2.1340, valid accuracy: 0.3972
Iter-23600 train loss: 2.1173 valid loss: 2.1334, valid accuracy: 0.3978
Iter-23700 train loss: 2.1691 valid loss: 2.1328, valid accuracy: 0.3976
Iter-23800 train loss: 2.1253 valid loss: 2.1322, valid accuracy: 0.3984
Iter-23900 train loss: 2.1064 valid loss: 2.1316, valid accuracy: 0.3984
Iter-24000 train loss: 2.1544 valid loss: 2.1310, valid accuracy: 0.3986
Iter-24100 train loss: 2.0923 valid loss: 2.1304, v

Iter-34100 train loss: 2.0646 valid loss: 2.0728, valid accuracy: 0.4252
Iter-34200 train loss: 2.0723 valid loss: 2.0722, valid accuracy: 0.4256
Iter-34300 train loss: 2.1272 valid loss: 2.0717, valid accuracy: 0.4258
Iter-34400 train loss: 2.0824 valid loss: 2.0711, valid accuracy: 0.4262
Iter-34500 train loss: 2.1188 valid loss: 2.0705, valid accuracy: 0.4268
Iter-34600 train loss: 2.1308 valid loss: 2.0700, valid accuracy: 0.4264
Iter-34700 train loss: 2.0626 valid loss: 2.0694, valid accuracy: 0.4266
Iter-34800 train loss: 2.0804 valid loss: 2.0688, valid accuracy: 0.4268
Iter-34900 train loss: 2.0647 valid loss: 2.0683, valid accuracy: 0.4266
Iter-35000 train loss: 2.0479 valid loss: 2.0677, valid accuracy: 0.4278
Iter-35100 train loss: 2.1354 valid loss: 2.0672, valid accuracy: 0.4280
Iter-35200 train loss: 2.0695 valid loss: 2.0666, valid accuracy: 0.4280
Iter-35300 train loss: 2.0874 valid loss: 2.0660, valid accuracy: 0.4280
Iter-35400 train loss: 2.0230 valid loss: 2.0655, v

Iter-45400 train loss: 1.9669 valid loss: 2.0120, valid accuracy: 0.4442
Iter-45500 train loss: 2.0118 valid loss: 2.0115, valid accuracy: 0.4436
Iter-45600 train loss: 1.9712 valid loss: 2.0110, valid accuracy: 0.4446
Iter-45700 train loss: 1.9797 valid loss: 2.0105, valid accuracy: 0.4448
Iter-45800 train loss: 2.0504 valid loss: 2.0100, valid accuracy: 0.4448
Iter-45900 train loss: 2.0240 valid loss: 2.0094, valid accuracy: 0.4456
Iter-46000 train loss: 2.0404 valid loss: 2.0089, valid accuracy: 0.4456
Iter-46100 train loss: 2.0406 valid loss: 2.0084, valid accuracy: 0.4452
Iter-46200 train loss: 1.9731 valid loss: 2.0079, valid accuracy: 0.4450
Iter-46300 train loss: 2.0059 valid loss: 2.0073, valid accuracy: 0.4452
Iter-46400 train loss: 2.0284 valid loss: 2.0068, valid accuracy: 0.4458
Iter-46500 train loss: 1.9869 valid loss: 2.0063, valid accuracy: 0.4458
Iter-46600 train loss: 1.9729 valid loss: 2.0058, valid accuracy: 0.4460
Iter-46700 train loss: 2.0259 valid loss: 2.0052, v

...

Iter-257200 train loss: 1.3664 valid loss: 1.2767, valid accuracy: 0.6052
Iter-257300 train loss: 1.3548 valid loss: 1.2764, valid accuracy: 0.6058
Iter-257400 train loss: 1.3057 valid loss: 1.2762, valid accuracy: 0.6052
Iter-257500 train loss: 1.5877 valid loss: 1.2759, valid accuracy: 0.6054
Iter-257600 train loss: 1.4149 valid loss: 1.2757, valid accuracy: 0.6054
Iter-257700 train loss: 1.2031 valid loss: 1.2755, valid accuracy: 0.6056
Iter-257800 train loss: 1.4905 valid loss: 1.2753, valid accuracy: 0.6058
Iter-257900 train loss: 1.4762 valid loss: 1.2750, valid accuracy: 0.6056
Iter-258000 train loss: 1.3566 valid loss: 1.2748, valid accuracy: 0.6058
Iter-258100 train loss: 1.2639 valid loss: 1.2746, valid accuracy: 0.6060
Iter-258200 train loss: 1.3911 valid loss: 1.2743, valid accuracy: 0.6062
Iter-258300 train loss: 1.1650 valid loss: 1.2741, valid accuracy: 0.6062
Iter-258400 train loss: 1.3546 valid loss: 1.2739, valid accuracy: 0.6060
Iter-258500 train loss: 1.2843 valid l

Iter-268300 train loss: 1.2894 valid loss: 1.2517, valid accuracy: 0.6128
Iter-268400 train loss: 1.3617 valid loss: 1.2515, valid accuracy: 0.6126
Iter-268500 train loss: 1.2609 valid loss: 1.2513, valid accuracy: 0.6128
Iter-268600 train loss: 1.1343 valid loss: 1.2511, valid accuracy: 0.6126
Iter-268700 train loss: 0.9938 valid loss: 1.2508, valid accuracy: 0.6128
Iter-268800 train loss: 1.3426 valid loss: 1.2506, valid accuracy: 0.6128
Iter-268900 train loss: 1.3441 valid loss: 1.2504, valid accuracy: 0.6126
Iter-269000 train loss: 0.9935 valid loss: 1.2502, valid accuracy: 0.6126
Iter-269100 train loss: 1.1478 valid loss: 1.2500, valid accuracy: 0.6130
Iter-269200 train loss: 1.1546 valid loss: 1.2497, valid accuracy: 0.6128
Iter-269300 train loss: 1.2147 valid loss: 1.2495, valid accuracy: 0.6128
Iter-269400 train loss: 1.0771 valid loss: 1.2493, valid accuracy: 0.6126
Iter-269500 train loss: 1.2119 valid loss: 1.2491, valid accuracy: 0.6126
Iter-269600 train loss: 1.1358 valid l

Iter-279400 train loss: 1.0207 valid loss: 1.2278, valid accuracy: 0.6186
Iter-279500 train loss: 1.0681 valid loss: 1.2276, valid accuracy: 0.6186
Iter-279600 train loss: 1.0075 valid loss: 1.2274, valid accuracy: 0.6188
Iter-279700 train loss: 1.4365 valid loss: 1.2272, valid accuracy: 0.6186
Iter-279800 train loss: 1.2199 valid loss: 1.2270, valid accuracy: 0.6184
Iter-279900 train loss: 1.1188 valid loss: 1.2268, valid accuracy: 0.6186
Iter-280000 train loss: 1.1769 valid loss: 1.2265, valid accuracy: 0.6186
Iter-280100 train loss: 1.2127 valid loss: 1.2263, valid accuracy: 0.6188
Iter-280200 train loss: 1.1960 valid loss: 1.2261, valid accuracy: 0.6190
Iter-280300 train loss: 1.1193 valid loss: 1.2259, valid accuracy: 0.6186
Iter-280400 train loss: 1.2159 valid loss: 1.2257, valid accuracy: 0.6188
Iter-280500 train loss: 1.0285 valid loss: 1.2255, valid accuracy: 0.6192
Iter-280600 train loss: 1.1589 valid loss: 1.2252, valid accuracy: 0.6190
Iter-280700 train loss: 1.2852 valid l

Iter-290500 train loss: 1.2970 valid loss: 1.2047, valid accuracy: 0.6252
Iter-290600 train loss: 1.2225 valid loss: 1.2045, valid accuracy: 0.6250
Iter-290700 train loss: 1.1849 valid loss: 1.2043, valid accuracy: 0.6250
Iter-290800 train loss: 1.1070 valid loss: 1.2041, valid accuracy: 0.6250
Iter-290900 train loss: 1.1757 valid loss: 1.2039, valid accuracy: 0.6256
Iter-291000 train loss: 1.2862 valid loss: 1.2037, valid accuracy: 0.6256
Iter-291100 train loss: 1.1087 valid loss: 1.2035, valid accuracy: 0.6256
Iter-291200 train loss: 1.1085 valid loss: 1.2033, valid accuracy: 0.6254
Iter-291300 train loss: 1.0943 valid loss: 1.2031, valid accuracy: 0.6260
Iter-291400 train loss: 1.3021 valid loss: 1.2029, valid accuracy: 0.6266
Iter-291500 train loss: 1.2595 valid loss: 1.2027, valid accuracy: 0.6270
Iter-291600 train loss: 1.1628 valid loss: 1.2024, valid accuracy: 0.6276
Iter-291700 train loss: 1.2394 valid loss: 1.2022, valid accuracy: 0.6276
Iter-291800 train loss: 1.1411 valid l

Iter-301600 train loss: 1.5001 valid loss: 1.1826, valid accuracy: 0.6300
Iter-301700 train loss: 1.3249 valid loss: 1.1825, valid accuracy: 0.6300
Iter-301800 train loss: 1.2507 valid loss: 1.1823, valid accuracy: 0.6300
Iter-301900 train loss: 1.1641 valid loss: 1.1821, valid accuracy: 0.6298
Iter-302000 train loss: 1.1359 valid loss: 1.1819, valid accuracy: 0.6296
Iter-302100 train loss: 1.2433 valid loss: 1.1817, valid accuracy: 0.6292
Iter-302200 train loss: 1.1700 valid loss: 1.1815, valid accuracy: 0.6296
Iter-302300 train loss: 1.1952 valid loss: 1.1813, valid accuracy: 0.6294
Iter-302400 train loss: 1.3622 valid loss: 1.1811, valid accuracy: 0.6290
Iter-302500 train loss: 1.2694 valid loss: 1.1809, valid accuracy: 0.6286
Iter-302600 train loss: 1.0791 valid loss: 1.1808, valid accuracy: 0.6288
Iter-302700 train loss: 1.3402 valid loss: 1.1806, valid accuracy: 0.6288
Iter-302800 train loss: 1.2321 valid loss: 1.1804, valid accuracy: 0.6292
Iter-302900 train loss: 1.1071 valid l

In [ ]:
# # Display the learning curve and losses for training, validation, and testing
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

plt.plot(nn.losses['train'], label='Train loss')
plt.plot(nn.losses['valid'], label='Valid loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(nn.losses['valid_acc'], label='Valid accuracy')
plt.legend()
plt.show()